In [10]:
import pandas as pd
import numpy as np
import re
import requests
import plotly
import html
import time

from bs4 import BeautifulSoup

In [11]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs

# I used Firefox; you can use whichever browser you like.
browser = webdriver.Chrome()

# Tell Selenium to get the URL you're interested in.
browser.get("https://twitter.com/hashtag/tsunami?lang=en")

# Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
        lastCount = lenOfPage
        time.sleep(1)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

# Now that the page is fully scrolled, grab the source code.
source_data = browser.page_source

# Throw your source into BeautifulSoup and start parsing!
bs_data = bs(source_data)

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")




In [12]:
#bs_data

In [13]:
#bs_data.find_all('p', {'class': 'TweetTextSize js-tweet-text tweet-text'})

In [14]:
str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[10]).split('href="/')[1].split('/status')[0]

'IND_center'

---

## Number of Tweets Captured

In [15]:
num_tweets = len(list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'})))

In [16]:
num_tweets

0

---

## Handle

In [17]:
str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('href="/')[1].split('/status')[0]

'doomalert19'

---

## Time

In [18]:
time_str = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('title="')[1].split('"><span ')[0].split(' - ')[0]

---

## Date

Day

In [19]:
day_str = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[0]

Month

In [20]:
month_str = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[1]

Year

In [21]:
year_str = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[2]

---

## Convert to datetime object

In [22]:
from datetime import datetime

#datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

In [23]:
time_str.replace(" ", "")

'10:21AM'

In [24]:
datetime_object = datetime.strptime(month_str + ' ' + day_str + ' ' + year_str + ' ' + time_str.replace(" ", ""), '%b %d %Y %I:%M%p')

---

## Tweet

In [28]:
#tweet = str(list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'}))[0]).split('lang="en">')[1].split('<a class')[0].rstrip()

---

## Check for keyword (tsunami)

In [26]:
tweet = 'This is a tweet about a #tsunami'

In [27]:
'#tsunami' in tweet

True

---

## Build Dictionary

In [119]:
temp_dict = {}

temp_dict['handle']     = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('href="/')[1].split('/status')[0]
temp_dict['tweet']      = str(list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'}))[0]).split('lang="en">')[1].split('<a class')[0].rstrip()
temp_dict['datetime']   = datetime_object
temp_dict['if_tsunami'] = 'tsunami' in temp_dict['tweet']

In [125]:
temp_dict

{'handle': 'thejakartaglobe',
 'tweet': 'China Pledges Easier Foreign Tourist Access to Tibet Amid US Pressure',
 'datetime': datetime.datetime(2019, 1, 13, 6, 25),
 'if_tsunami': False}

In [28]:
bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize js-tweet-text tweet-text'})

[]

In [37]:
master_list = []
#loop_range = num_tweets - 1
loop_range = 10


for i in range(loop_range):
    tweets = bs_data.find_all('p', {'class': 'TweetTextSize js-tweet-text tweet-text'})

    temp_dict = {}

    temp_dict['handle']     = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[i]).split('href="/')[1].split('/status')[0]
    
    try:
        temp_dict['tweet']      = str(tweets[i].get_text()).replace("\n", " ")
        #temp_dict['tweet']      = str(list(bs_data.find_all('p', {bs_data.find_all('p', {'class': 'TweetTextSize js-tweet-text tweet-text'})[i].split('lang="en">')[1].split('<a class')[0].rstrip()
    except IndexError:
        temp_dict['tweet']      = ""
    
    temp_dict['day']        = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[i]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[0]
    temp_dict['month']      = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[i]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[1]
    temp_dict['year']       = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[i]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[2]
    temp_dict['time']       = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[i]).split('title="')[1].split('"><span ')[0].split(' - ')[0]
    
    datetime_object = datetime.strptime(temp_dict['month'] + ' ' + temp_dict['day'] + ' ' + temp_dict['year'] + ' ' + temp_dict['time'].replace(" ", ""), '%b %d %Y %I:%M%p')
    
    temp_dict['datetime']   = datetime_object
    temp_dict['if_tsunami'] = 'tsunami' in temp_dict['tweet'] or 'Tsunami' in tweet or '#tsunami' in tweet or '#Tsunami' in tweet
    
    master_list.append(temp_dict)
    print(i)
    

0
1
2
3
4
5
6
7
8
9


In [38]:
master_list[1]

{'handle': 'SSkyvington',
 'tweet': '#Tsunami  Three health care tsunamis — obesity, diabetes, and dementia — are waiting for us, for boomers and zoomers especially, looming just on the horizon, building up speed, and threatening to swamp our health care system and wipe out the treasury.  https://www.dundurn.com/books/May-Hurt-Bit\xa0…https://twitter.com/NeilFlochMD/status/1084796525509857281\xa0…',
 'day': '14',
 'month': 'Jan',
 'year': '2019',
 'time': '5:00 AM',
 'datetime': datetime.datetime(2019, 1, 14, 5, 0),
 'if_tsunami': True}

In [174]:
pd.DataFrame(master_list).to_csv('thejakartaglobe_tweetscrape.csv', sep=",", index=False)

In [175]:
tweet = 'this is a tweet with Tsunami in it'

In [176]:
'tsunami' in tweet or 'Tsunami' in tweet

True

In [33]:
str(list(bs_data.find_all('p', {bs_data.find_all('p', {'class': 'TweetTextSize js-tweet-text tweet-text'})[0].split('lang="en">')[1].split('<a class')[0].rstrip()

SyntaxError: unexpected EOF while parsing (<ipython-input-33-46f27fef284e>, line 1)

In [43]:
str(list(bs_data.find_all('p', {'class': 'TweetTextSize js-tweet-text tweet-text'}))[9]).split('lang="en">')[1].split('<a class')[0].rstrip()

'Are'

In [48]:
str(list(bs_data.find_all('p', {'class': 'TweetTextSize js-tweet-text tweet-text'}))[8])

'<p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="en">Real-time updates <a class="twitter-atreply pretty-link js-nav" data-mentioned-user-id="1006255208379211776" dir="ltr" href="/DisasterawareE"><s>@</s><b>DisasterAWAREE</b></a> for 78 currently on-going active natural hazards around the world! <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/highsurf?src=hash"><s>#</s><b>highsurf</b></a> <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/WinterStorm?src=hash"><s>#</s><b>WinterStorm</b></a> <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/flood?src=hash"><s>#</s><b>flood</b></a> <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/earthquake?src=hash"><s>#</s><b>earthquake</b></a> <a class="twitter-hashtag pretty-link js-nav" data-query-so

In [149]:
range(loop_range)

range(0, 579)

In [153]:
str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[579]).split('href="/')[1].split('/status')[0]

'thejakartaglobe'

In [155]:
i

5

In [ ]:
str(list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'}))[5]).split('lang="en">')[1].split('<a class')[0].rstrip()